In [ ]:
!pip install google-cloud-bigquery google-auth db-dtypes

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')
import pandas as pd

In [2]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('./multicampus_project_first_account.json')

In [3]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project = credentials.project_id
)

===============================================================================================================================================================================================================================

In [6]:
# 1번 : 자동차 등록 수 자료
# 기준 : 2023년 10월 31일

sql = '''
SELECT
  Year,
  Total
FROM
  project_1.Registration_All
'''

In [8]:
df = client.query(sql).to_dataframe()

In [ ]:
sns.barplot(
    data = df,
    x = 'Year',
    y = 'Total'
    ).set(
        title = 'The Number of Registered Vehicles'
        )
plt.xlabel('Year')   # https://wikidocs.net/92081  x축,y축 레이블 설정
plt.ylabel('')


In [11]:
# 상승률 계산


answer = []
num = df['Total']

for i in range(len(df['Total'])-1):
    answer.append(round((num[i+1]-num[i])/num[i]*100,2))
print(answer)
print(sum(answer)/len(answer))

[2.89, 3.44, 4.13, 4.03, 3.43, 3.01, 2.18, 2.78, 2.31, 2.39, 1.84]
2.9481818181818182


In [ ]:
years = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
plt.title('The Annual Growth Rate of Registered Vehicles')
plt.plot(years,answer)
plt.xlim(2012,2024)
plt.ylim(0,5)
plt.show()

# 그래프 그리기 : https://wikidocs.net/92082

===============================================================================================================================================================================================================================

In [14]:
# 2번 : 전기차 등록 수 자료
# 기준 : 2023년 10월 31일

sql = '''
SELECT
  Year,
  Total
FROM
  project_1.Registration_EV
'''

In [15]:
df = client.query(sql).to_dataframe()

In [ ]:
plt.xticks(rotation=45)
sns.barplot(
    data = df,
    x = 'Year',
    y = 'Total'
    ).set(
        title = 'The Number of Registered Electric Vehicles'
        )
plt.xlabel('Year')
_=plt.ylabel('')

In [ ]:
# 상승률 계산


answer = []
num = df['Total']

for i in range(len(df['Total'])-1):
    answer.append(round((num[i+1]-num[i])/num[i]*100,2))
print(answer)
print(sum(answer)/len(answer))

In [ ]:
years = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
plt.title('The Annual Growth Rate of EV Registrations')
plt.plot(years,answer)
plt.xlim(2012,2024)
plt.ylim(0,300)
plt.show()

===============================================================================================================================================================================================================================

In [34]:
# 3  번 : 전체 충전소 vs 고속도로 충전소 수 비교
# 고속도로 휴게소 충전소 수

sql = '''
SELECT
  COUNT(Facility_2)
FROM
  project_1.ChargingStations
WHERE
  Facility_2 LIKE '%휴게소%'
'''

In [ ]:
df = client.query(sql).to_dataframe()

In [30]:
# 전체 충전소 수
sql = '''
SELECT
  COUNT(Facility_2)
FROM
  project_1.ChargingStations
'''

In [ ]:
df = client.query(sql).to_dataframe()

In [ ]:
ratio = [99.48,0.52]
labels = ['Total','Highway']
colors = ['skyblue','green']

plt.pie(ratio,labels = labels, autopct = '%.1f%%',startangle = 130,colors=colors)
plt.title('The Number of Registered Charging Stations')
plt.show()


===============================================================================================================================================================================================================================

In [13]:
# 4 - 1 번 : 고속도로 평균 충전량
sql = '''
SELECT
  COUNT(Type) AS number,
  Type,
  AVG(Amount) AS amount,
  AVG(TIMESTAMP_DIFF(End_Time,Start_Time,MINUTE)) AS time
FROM
  `apt-rite-404708.project_1.ChargingData_2023*`
WHERE
  Station_Name LIKE '%휴게소%'
GROUP BY
  Type
'''

In [14]:
df = client.query(sql).to_dataframe()
df.columns = ['충전기수', '타입', '충전량', '충전시간']

In [15]:
# 4 - 2 번 : 고속도로 충전량 

sql = '''
SELECT
  Type,
  Amount
FROM
  `apt-rite-404708.project_1.ChargingData_2023*`
WHERE
 Station_Name LIKE '%휴게소%'
'''

df = client.query(sql).to_dataframe()

In [ ]:
plt.figure(figsize=(7, 5))
sns.histplot(
    data = df,
    x = 'Amount',
    bins=[x for x in range(0,100,2)]
    ).set(
        title = 'Average Charging Capacity at Highway Rest Area'
    )

In [17]:
# 4 - 3 번 : 고속도로 충전시간

sql = '''
SELECT
  Type,
  TIMESTAMP_DIFF(End_Time,Start_Time,MINUTE) AS time
FROM
  `apt-rite-404708.project_1.ChargingData_2023*`
WHERE
 Station_Name LIKE '%휴게소%'
'''

df = client.query(sql).to_dataframe()

In [ ]:
plt.figure(figsize=(7, 5))
sns.histplot(
    data = df,
    x = 'time',
    bins=[x for x in range(0,100,2)]
    ).set(
        title = 'Average Charging Time at Highway Rest Area'
    )

===============================================================================================================================================================================================================================

In [9]:
# 5번 : 고속도로 통행량

sql = '''
SELECT
  Route_Name,
  SUM(Traffic_Volume) AS traffic
FROM
  project_1.Traffic_2023_11_12
GROUP BY
  Route_Name
ORDER BY
  traffic DESC
LIMIT
 15
'''

In [10]:
df = client.query(sql).to_dataframe()

In [ ]:
plt.figure(figsize=(6, 5))
sns.barplot(
    data = df,
    x = 'traffic',
    y = 'Route_Name'
).set(
    
    title = 'Top 15 Highway Traffic Volume'
)
plt.xlabel('Traffic Volume')
_=plt.ylabel('Highway Name')

===============================================================================================================================================================================================================================

In [4]:
# 6번 : 고속도로 통행량
# 기준 : 2022년 일평균


sql = '''
SELECT
  Section,
  SUM(DailyAverage) AS DA
FROM
  project_1.Traffic_2022
GROUP BY
  Section
ORDER BY
  SUM(DailyAverage) DESC
LIMIT
  15
'''

In [5]:
df = client.query(sql).to_dataframe()

In [ ]:
plt.figure(figsize=(6, 5))
sns.barplot(
    data=df,
    y='Section',
    x='DA'
).set(
    title = 'Top 15 Daily Highway Segment Traffic Volume'
)
plt.xlabel('Traffic Volume')
plt.ylabel('Highway Segment')

===============================================================================================================================================================================================================================

In [81]:
# 7번 : 휴게소 이용객
# 기준 : 23년 8월 24일

sql = '''
SELECT
  Rest_Area_Name,
  Daily_Visitors
FROM
  project_1.Traffic_RestArea
WHERE
  Daily_Visitors IS NOT NULL
ORDER BY
  Daily_Visitors DESC
LIMIT
  15
'''

In [82]:
df = client.query(sql).to_dataframe()

In [ ]:
barplot = sns.barplot(
    data = df,
    y = 'Rest_Area_Name',
    x = 'Daily_Visitors'
).set(
    
    title = 'Top 15 Rest Area Visitors'
)

plt.xlabel('Number of Visitors')
plt.ylabel('Rest Area Name')

barplot

===============================================================================================================================================================================================================================

Site Selection

고속도로 노선 교통량 상위 10개 -> 그중 노선 상위 10개 -> 그중 휴게소 방문대수 상위 10개

===============================================================================================================================================================================================================================

In [ ]:
# 1 : 고속도로 노선 교통량 상위 5개
sql = '''
SELECT
  Route_Name,
  SUM(Traffic_Volume) AS traffic
FROM
  project_1.Traffic_2023_11_12
GROUP BY
  Route_Name
ORDER BY
  traffic DESC
LIMIT
 5
'''
Highway = client.query(sql).to_dataframe()
Highway

In [ ]:
sql = '''
SELECT
  Rest_Area_Name,
  Daily_Visitors
FROM
  project_1.Traffic_RestArea
WHERE
 Route_Name IN ('경부선')
ORDER BY
  Daily_Visitors DESC
LIMIT
 5
'''
Route_1 = client.query(sql).to_dataframe()
Route_1

In [ ]:
sql = '''
SELECT
  Rest_Area_Name,
  Daily_Visitors
FROM
  project_1.Traffic_RestArea
WHERE
 Route_Name IN ('영동선')
ORDER BY
  Daily_Visitors DESC
LIMIT
 5
'''
Route_2 = client.query(sql).to_dataframe()
Route_2

In [ ]:
sql = '''
SELECT
  Rest_Area_Name,
  Daily_Visitors
FROM
  project_1.Traffic_RestArea
WHERE
 Route_Name IN ('중부선')
ORDER BY
  Daily_Visitors DESC
LIMIT
 5
'''
Route_3 = client.query(sql).to_dataframe()
Route_3

In [ ]:
sql = '''
SELECT
  Rest_Area_Name,
  Daily_Visitors
FROM
  project_1.Traffic_RestArea
WHERE
 Route_Name IN ('남해선')
ORDER BY
  Daily_Visitors DESC
LIMIT
 5
'''
Route_4 = client.query(sql).to_dataframe()
Route_4

In [ ]:
sql = '''
SELECT
  Rest_Area_Name,
  Daily_Visitors
FROM
  project_1.Traffic_RestArea
WHERE
 Route_Name IN ('서해안선')
ORDER BY
  Daily_Visitors DESC
LIMIT
 5
'''
Route_5 = client.query(sql).to_dataframe()
Route_5

In [ ]:
Ranking = pd.DataFrame(columns = Highway['Route_Name'])
Ranking['경부선'] = Route_1['Rest_Area_Name']
Ranking['영동선'] = Route_2['Rest_Area_Name']
Ranking['중부선-대전통영선A'] = Route_3['Rest_Area_Name']
Ranking['남해선A'] = Route_4['Rest_Area_Name']
Ranking['서해안선'] = Route_5['Rest_Area_Name']

Ranking

#  https://zephyrus1111.tistory.com/33 판다스 표 만들기

끝